In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
from scipy.stats import linregress

from config import weather_api_key
from citipy import citipy
from datetime import datetime


In [2]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs


In [4]:
coordinates = list(lat_lngs)


In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)


727

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ", "+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(
            city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

# Loop through all the cities in our list.
for i in range(len(cities)):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + cities[i]


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | mugur-aksy
Processing Record 2 of Set 1 | thompson
Processing Record 3 of Set 1 | cape town
Processing Record 4 of Set 1 | ginda
Processing Record 5 of Set 1 | hilo
Processing Record 6 of Set 1 | ushuaia
Processing Record 7 of Set 1 | mataura
Processing Record 8 of Set 1 | busselton
Processing Record 9 of Set 1 | bireun
Processing Record 10 of Set 1 | rikitea
Processing Record 11 of Set 1 | korla
Processing Record 12 of Set 1 | kahului
Processing Record 13 of Set 1 | pangnirtung
Processing Record 14 of Set 1 | punta arenas
Processing Record 15 of Set 1 | tezu
Processing Record 16 of Set 1 | nikolskoye
Processing Record 17 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 18 of Set 1 | uzhur
Processing Record 19 of Set 1 | kapaa
Processing Record 20 of Set 1 | yar-sale
Processing Record 21 of Set 1 | atuona
Processing Record 22 of Set 1 | jamestown
Processing Record 23 of Set 1 

In [7]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mugur-Aksy,50.3500,90.5000,-8.91,69,10,6.15,RU,2022-01-30 01:17:35
1,Thompson,55.7435,-97.8558,-12.84,77,20,0.00,CA,2022-01-30 01:14:51
2,Cape Town,-33.9258,18.4232,70.21,69,0,5.01,ZA,2022-01-30 01:17:19
3,Ginda,30.6945,78.4932,37.72,30,0,8.08,IN,2022-01-30 01:17:36
4,Hilo,19.7297,-155.0900,76.80,69,0,7.00,US,2022-01-30 01:17:27
5,Ushuaia,-54.8000,-68.3000,44.26,87,100,0.00,AR,2022-01-30 01:12:57
6,Mataura,-46.1927,168.8643,56.73,99,100,2.64,NZ,2022-01-30 01:13:02
7,Busselton,-33.6500,115.3333,77.09,60,7,13.00,AU,2022-01-30 01:17:16
8,Bireun,5.2030,96.7009,77.22,83,99,3.38,ID,2022-01-30 01:17:36
9,Rikitea,-23.1203,-134.9692,76.98,71,100,8.48,PF,2022-01-30 01:17:13


In [8]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")
